<a href="https://colab.research.google.com/github/mparag019/DL-Assignment-2/blob/main/PART-A/DL_Assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4725179%2F8019269%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T120039Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77e7021331966dd64f9bb54bcea7aba4b233a4210254cad096241cd8cd556def65272ef8426e288e142db8e3e53a24e2e26c5185bf772aea0688711204bfe1d8e23c752c353a90ebb0ea5e70359f27e80687e5d664b7a47b8182d55b0c8b436e0394a899fe7bdac3694380e4fb228a24a7e25578b2c77462f7eeacd25061d7b3226eb858e937255a0967165f726557accbdf254e89640430e6ecc56ab86c9edb9727ed8e9f37c565aba031ea8ff96a914760e1b7beac34bf5fb2cef3c046a5cf60140fe035efd4f8a5fe3fca026a570dc573e20a3bb4bf377b97cbe076c04652422a9cec87286ea518377009353b38295b18c6b7a94aeb9d4b111d8966ce1e29'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install torch torchvision wandb


# **Question 1 & Question 2**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import wandb
from wandb.sdk.wandb_run import Run
from tqdm import tqdm

# Initialize Wandb
wandb.init(project='DL-Assignment-2', entity='cs23m047')

# Define hyperparameters for hyperparameter tuning
config = wandb.config
config.filters = 32 # Example values for number of filters
config.activation = 'ReLU'  # Example activation function
config.filter_organization = 'half'  # Example filter organization
config.data_augmentation = False  # Example data augmentation
config.batch_norm = True  # Example batch normalization
config.dropout = 0.2  # Example dropout rate

# Set up data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# # Load the dataset
train_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/train', transform=train_transforms)
test_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/val', transform=val_transforms)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

# Set up data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, filters, activation, filter_organization, data_augmentation, batch_norm, dropout):
        super(CNN, self).__init__()

        self.conv_layers = nn.Sequential()
        self.dense_layers = nn.Sequential()

        # Add conv layers based on filter organization
        if filter_organization == 'same':
            num_filters = [filters] * 5  # Assuming 3 convolution layers
        elif filter_organization == 'double':
            num_filters = [filters * 2**i for i in range(5)]  # Doubling filters in each subsequent layer
        else:
            num_filters = [filters // 2**i for i in range(5)]  # Halving filters in each subsequent layer

        in_channels = 3
        for i, f in enumerate(num_filters):
            print(f)
            self.conv_layers.add_module(f"conv_{i}", nn.Conv2d(in_channels, f, kernel_size=3, padding=1))
            if batch_norm:
                self.conv_layers.add_module(f"batch_norm_{i}", nn.BatchNorm2d(f))
            self.conv_layers.add_module(f"{activation}_{i}", getattr(nn, activation)())
            self.conv_layers.add_module(f"maxpool_{i}", nn.MaxPool2d(kernel_size=2))
            if data_augmentation and i == 0:
                self.conv_layers.add_module("dropout", nn.Dropout2d(p=dropout))  # Dropout only in the first layer
            in_channels = f

        self.dense_layers.add_module("flatten", nn.Flatten())
        self.dense_layers.add_module("dense", nn.Linear(in_channels * 7 * 7, 512))
        self.dense_layers.add_module("relu", nn.ReLU())
        self.output_layer = nn.Linear(512, len(train_data.classes))

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.dense_layers(x)
        x = self.output_layer(x)
        return x


# Create an instance of the CNN model
model = CNN(config.filters, config.activation, config.filter_organization,
            config.data_augmentation, config.batch_norm, config.dropout)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    training_loss = 0.0
    train_accuracy = 0.0
    pbar = tqdm(train_loader, total=len(train_loader))
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        train_accuracy += calculate_accuracy(outputs, labels)


    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            val_accuracy += calculate_accuracy(outputs, labels)

    train_accuracy /= len(train_loader)
    training_loss /= len(train_loader)
    val_loss /= len(val_loader)
    val_accuracy /= len(val_loader)
    pbar.set_description(f'Epoch {epoch+1}/{epochs}, Train_Loss: {training_loss:.4f},  Train_Acc: {train_accuracy:.4f},  Val_Loss: {val_loss:.4f},  Val_Loss: {val_accuracy:.4f}')

    wandb.log({"epoch": epoch+1, "train_loss": training_loss, "val_loss": val_loss, "val_accuracy": val_accuracy, "train_accuracy": train_accuracy})

# Test the model
# model.eval()
# test_accuracy = 0.0
# with torch.no_grad():
#     for images, labels in test_loader:
#         outputs = model(images)
#         test_accuracy += calculate_accuracy(outputs, labels)

# test_accuracy /= len(test_loader)
# wandb.log({"test_accuracy": test_accuracy})

# Save the model
# torch.save(model.state_dict(), 'model.pth')
